Inspiration for Neural Net: https://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html

# Data Pre-Processing

In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pitcher
from random import shuffle
torch.manual_seed(1)

In [40]:
player = 'lester'
data = pd.read_csv('Data/raw_data/'+player+'.csv')
data = pitcher.clean_data(data)
ABs = pitcher.get_abs(data)
reps = pitcher.get_reps(ABs)
reps = pitcher.drop_nas(reps)

In [70]:
len(ABs1[0][0])

46

In [95]:
player = 'verlander'
data = pd.read_csv('Data/raw_data/'+player+'.csv')
data = pitcher.clean_data(data)
ABs = pitcher.get_abs(data)
reps = get_reps(ABs)

In [76]:
ABs[0]

[array([430203, 'CH', -2.553030833333333, 1.0261733333333332, 84.2, -0.512,
        1.932, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       dtype=object),
 array([430203, nan, nan, nan, nan, nan, nan, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1,
        0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0], dtype=object),
 array([430203, nan, nan, nan, nan, nan, nan, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0,
        1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0], dtype=object),
 array([115210, 'CH', -0.8984783333333333, -0.2644, 82.5, 0.682, 1.455, 0,
        1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0], dtype=object)]

In [88]:
reps[0]

([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [-2.553030833333333,
   1.0261733333333332,
   84.2,
   -0.512,
   1.932,
   1,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   1,
   0,
   0,
   0],
  [nan,
   nan,
   nan,
   nan,
   nan,
   0,
   0,
   1,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0],
  [nan,
   nan,
   nan,
   nan,
   nan,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   1,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0],
  [-0.8984783333333333,
   -0.2644,
   82.5,
   0.682,
   1.455,
   0,
   0,
   1,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   1,
   0,
   0,
   0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
 [[1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0],
  [1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0],
  [1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0,

In [107]:
len(reps)

9776

In [106]:
good_reps = []
for rep in reps:
    is_na = False
    prevs,states,pitches = rep
    for prev in prevs:
        for var in prev:
            if var != var:
                is_na = True
    if not is_na:
        good_reps.append(rep)
        is_na = False
reps = good_reps             

In [86]:
def get_reps(pitcher_ABs):
    reps = []
    for AB in pitcher_ABs:
        prev_0 = [0]*25
        ptypes = []
        for pitch in AB:
            if pitch[1] != 'FF':
                ptypes.append('NF') 
            else:
                ptypes.append('FF')

        prev_pitch_cont = [pitch[2:7].tolist() for pitch in AB]
        prev_pitch_disc = [pitch[22:38].tolist() for pitch in AB]
        ptypes_ = [pitch[38:42].tolist() for pitch in AB]
        pitches = zip(prev_pitch_cont,prev_pitch_disc)
        ###pitches = zip(prev_pitch_cont,ptypes_)
        prevs = [cont + disc for cont,disc in pitches]
        #prevs = prev_pitch_cont
        pitches_ = zip(prevs,ptypes_)
        ###prev_pitches = [prev + ptype for prev,ptype in pitches]
        prev_pitches = [prev + ptype for prev,ptype in pitches_]
        pre_pitch = [pitch[7:22].tolist() for pitch in AB]
        #prev_p = zip(prev_pitches_,pre_pitch)
        #prev_pitches = [prev + prepitch for prev,prepitch in prev_p]
        AB_len = len(AB)
        if AB_len <= 6:
            prevs = [prev_0] + prev_pitches + ([prev_0]*(6-(AB_len)-1))
            states = pre_pitch + ([[0]*15]*(6-AB_len))
            pitches = ptypes + (['NF']*(6-AB_len))
        else:
            prevs = [prev_0] +[prev_pitches[1:7]]
            states = [[0]*15]+[pre_pitch[:7]]
            pitches = ['NF'] + [ptypes[:7]]
        reps.append((prevs,states,pitches))
    return reps

In [41]:
len(data)

36986

In [3]:
reps = reps1

In [67]:
avg = 0
for AB in ABs:
    if len(AB) <= 6:
        avg += 1
print(avg/len(ABs))

0.90876029521874


In [60]:
#get train, validate, and test sets (60-20-20)
shuffle(reps)
cutoff1 = int(len(reps)*0.6)
cutoff2 = cutoff1 + int(len(reps)*.2)
train = reps[1:cutoff1]
validate = reps[cutoff1:cutoff2]
test = reps[cutoff2:]


#get train, validate, and test batches
batch_size = 50
train_batches = pitcher.get_batches(train,batch_size)
test_batches = pitcher.get_batches(test,batch_size)
validate_batches = pitcher.get_batches(validate,batch_size)

# Model Construction

In [5]:
def test_accuracy(batches,model):
    length = len(batches)*len(batches[0][0])
    num_right = 0
    ch_count = 0
    predict_ch = 0
    cu_count = 0
    predict_cu = 0
    sl_count = 0
    predict_sl = 0
    ff_count = 0
    predict_ff = 0
    ff_right = 0
    ch_right = 0
    sl_right = 0
    cu_right = 0
    for batch in batches:
        with torch.no_grad():
            prev_pitches,pre_pitch, ptypes = batch
            prevs_in = torch.tensor(prev_pitches, dtype=torch.float)
            rep_in = torch.tensor(pre_pitch, dtype=torch.float)
            #targets = tag_to_ix[ptypes]
            if len(ptypes) > 1:
                targets = [tag_to_ix[ptype] for ptype in ptypes]
            #else:
            #    targets = tag_to_ix[ptypes[0]]
            if torch.cuda.is_available():
                tag_scores = model((prevs_in,rep_in)).cuda()
            else:
                tag_scores = model((prevs_in,rep_in))
            #print(tag_scores)
            preds = [tag_score.max(0) for tag_score in tag_scores]
            #print(index.item())
            if len(ptypes) > 1:
                for pred,target in zip(preds,targets):
                    _,index = pred
                    if target == 1:
                        ff_count += 1
                    if index.item() == 1:
                        predict_ff += 1
                    if target == 1 and index.item() == 1:
                        ff_right += 1
                    if target == 0:
                        cu_count += 1
                    if index.item() == 0:
                        predict_cu += 1
                    if target == 0 and index.item() == 0:
                        cu_right += 1
                    if index.item() == target:
                        num_right += 1
            else:
                _,index = preds[0]
                if targets == 1:
                    ff_count += 1
                if index.item() == 1:
                    predict_ff += 1
                if targets == 1 and index.item() == 1:
                    ff_right += 1
                if targets == 0:
                    cu_count += 1
                if index.item() == 0:
                    predict_cu += 1
                if targets == 0 and index.item() == 0:
                    cu_right += 1
                if index.item() == targets:
                    num_right += 1
    ff_rate = ff_count/length
    pred_ff_rate = predict_ff/length
    ff_acc = ff_right/ff_count
    cu_rate = cu_count/length
    pred_cu_rate = predict_cu/length
    cu_acc = cu_right/cu_count
    accuracy = num_right/length
    print("______________________________________")
    print("Non-Fastball rate:",cu_rate)
    print("Predicted Non-Fastball rate:",pred_cu_rate)
    print("Non-Fastball accuracy:",cu_acc)
    print("______________________________________")
    print("Fastball rate:",ff_rate)
    print("Predicted Fastball rate:",pred_ff_rate)
    print("Fastball accuracy:",ff_acc)
    print("______________________________________")
    print("Accuracy:",accuracy)
    print("Accuracy above naive guess:",accuracy - ff_rate)
    print("______________________________________")

In [119]:
#change this for different pitcher
'''tag_to_ix = {'CU':0,
             'FF':1,
             'SL':2,
             'CH':3}'''
tag_to_ix = {'NF':0,
             'FF':1}

# -- Input Dimensions -- DO NOT CHANGE
PREV_PITCH_DIM = 25
NUM_PREV_PITCHES = 6
GAME_STATE_DIM = 15 
GAME_OUT_DIM = 15

# -- Hyperparameters -- DO CHANGE 
HIDDEN_DIM = 120
OUT_DIM = 10
lstm_layers = 1
learning_rate = 0.0001

if torch.cuda.is_available():
    torch.set_default_tensor_type(torch.cuda.FloatTensor)
    
class PitchPredict(nn.Module):
    def __init__(self, prev_pitch_dim, hidden_dim, num_prev_pitches,out_dim, game_state_dim, game_out_dim, num_ptypes):
        super(PitchPredict, self).__init__()
        #get constants
        self.hidden_dim = hidden_dim
        self.prev_pitch_dim = prev_pitch_dim
        self.num_prev_pitches = num_prev_pitches
        self.out_dim = out_dim
        self.game_state_dim = game_state_dim
        self.game_out_dim = game_out_dim
        
        ####Define Layers####
        
        ########################## LSTM for past five pitches#######################################
        # The LSTM takes previous pitch vectors as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.RNN(prev_pitch_dim, hidden_dim,num_layers=lstm_layers,batch_first=True)#FIDDLE WITH num_layers

        # The linear layer that maps from hidden state space to a vector
        # with dimensionality OUT_DIM
        self.hidden2out = nn.Linear(hidden_dim, out_dim)
        
        ############## FULLY CONNECTED LAYERS for LSTM OUTPUT + game_state vector #################
        
        # layer to map the game state vector to a different dimension
        self.l1 = nn.Linear(self.game_state_dim, self.game_out_dim)  
        
        # This Fully connected layer maps from the output of the final hidden layer output from the LSTM,
        # dimension = OUT_DIM, with the game state vector, dimension = GAME_STATE_DIM
        # to a vector of length of the number of ptypes to pass through softmax for probabilities
        self.fc1 = nn.Linear((self.out_dim + self.game_out_dim), len(tag_to_ix))

                             
    def forward(self, rep):
        past_pitches,game_states = rep
        #print(past_pitches)
        lstm_out, _ = self.lstm(past_pitches.view(len(past_pitches), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        #lstm_out, _ = self.lstm(past_pitches.view(batch_size,self.num_prev_pitches, -1))
        #learned_rep = self.hidden2out(lstm_out.view(batch_size,self.num_prev_pitches, -1))
        #game_rep = game_state.view(batch_size,self.game_state_dim)
        #game_rep = F.relu(self.l1(game_state.view(batch_size,self.game_state_dim)))
        #encoding = learned_rep[:,self.num_prev_pitches - 1:,:]
        #fc_in = torch.cat((encoding.view(batch_size,self.out_dim),game_rep.view(batch_size,self.game_out_dim)),dim=1)
        #print(learned_rep)
        #fc = self.fc1(fc_in.view(batch_size,self.game_out_dim+self.out_dim))
        #tag_scores = F.log_softmax(fc,dim=0)
        return tag_scores

In [125]:
reps[0][0]

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [[nan,
   nan,
   nan,
   nan,
   nan,
   0,
   0,
   1,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0],
  [-2.3815775,
   1.45216,
   95.1,
   -0.677,
   4.144,
   1,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0],
  [-2.4940241666666667,
   0.7893866666666666,
   83.6,
   -0.698,
   3.351,
   0,
   0,
   0,
   1,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   1,
   0,
   0,
   0],
  [-2.5510825,
   1.19674,
   83.6,
   -0.738,
   2.372,
   0,
   0,
   0,
   1,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   1,
   0,
   0,
   0],
  [nan,
   nan,
   nan,
   nan,
   nan,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   1,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0],
  [nan,
   nan,
   nan,
   nan,
   nan,
   0,


# Testing

# Training

In [120]:
model = PitchPredict(PREV_PITCH_DIM, HIDDEN_DIM, NUM_PREV_PITCHES, OUT_DIM, GAME_STATE_DIM, GAME_OUT_DIM, len(tag_to_ix))
loss_function = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# See what the scores are before training
# Note that element i,j of the output is the score for tag j for word i.
# Here we don't need to train, so the code is wrapped in torch.no_grad()
with torch.no_grad():
    print("***************Pre-Training Accuracy*****************")
    test_accuracy(train_batches,model)
print("*************************Training*****************************")
for epoch in range(1):
    #shuffle(train_batches)
    for batch in reps:
        prev_pitches,pre_pitch,ptypes = batch
        
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        #get input tensors ready
        prevs_in = torch.tensor(prev_pitches, dtype=torch.float)
        game_state_in = torch.tensor(pre_pitch, dtype=torch.float)
        
        #get target value
        #target = tag_to_ix[ptype]
        target = [ tag_to_ix[ptype] for ptype in ptypes]

        # Step 3. Run our forward pass.
        if torch.cuda.is_available():
            tag_scores = model((prevs_in,game_state_in)).cuda()
        else:
            tag_scores = model((prevs_in,game_state_in))
               
        # Step 4. Compute the loss, gradients, and update the parameters by
        #  calling optimizer.step()
        loss = loss_function(tag_scores, torch.tensor(target,dtype=torch.long))
        loss.backward()
        optimizer.step()
        
        preds = [tag_score.max(0) for tag_score in tag_scores]

                
    #display post-epoch results
    print('epoch:',epoch+1,"loss:",loss.item())

# See what the scores are after training
with torch.no_grad():
    print("****************Post-Training Accuracy********************")
    test_accuracy(train_batches,model.eval())
    print("**********************************************************")

***************Pre-Training Accuracy*****************


RuntimeError: input.size(-1) must be equal to input_size. Expected 25, got 75

# Results

In [56]:
with torch.no_grad():
    print("****************Validation Accuracy********************")
    test_accuracy(validate_batches,model)
    print("**********************************************************")

****************Validation Accuracy********************
______________________________________
Non-Fastball rate: 0.5553424657534246
Predicted Non-Fastball rate: 0.5436986301369863
Non-Fastball accuracy: 0.6250616674888998
______________________________________
Fastball rate: 0.44465753424657534
Predicted Fastball rate: 0.4563013698630137
Fastball accuracy: 0.5579174368453481
______________________________________
Accuracy: 0.5952054794520548
Accuracy above naive guess: 0.15054794520547948
______________________________________
**********************************************************


In [32]:
with torch.no_grad():
    print("****************Test Accuracy********************")
    test_accuracy(test_batches,model)
    print("**********************************************************")

****************Test Accuracy********************
______________________________________
Non-Fastball rate: 0.41554140127388534
Predicted Non-Fastball rate: 0.5212738853503185
Non-Fastball accuracy: 0.6661557326793378
______________________________________
Fastball rate: 0.5844585987261146
Predicted Fastball rate: 0.4787261146496815
Fastball accuracy: 0.5817349607672189
______________________________________
Accuracy: 0.6168152866242038
Accuracy above naive guess: 0.03235668789808921
______________________________________
**********************************************************


### high score using balanced set (25% FF)
- **TRAINING ACCURACY HIGH SCORE:** 38 (39% FF)

- **TEST ACCURACY HIGH SCORE:** 43.7 (37% FF) -14%- naive 


- ****HYPERPARAMETERS:****

    - LSTM num_layers: 1
    
    - OUT_SIZE: 15
    
    - HIDDEN_DIMENSION: 10
    
    - EPOCHS: 5
   
    - LEARNING RATE: 0.001
    
    - LOSS FUNCTION: NLLLOSS
    
    - OPTIMIZER: ADAM

### high score using unbalanced set (50+% FF)
- **TRAINING ACCURACY HIGH SCORE:** 57.8 (91% FF)

- **TEST ACCURACY HIGH SCORE:** 60.86 (88% FF) 2.5%+ naive 


- ****HYPERPARAMETERS:****

    - LSTM num_layers: 1
    
    - OUT_SIZE: 15
    
    - HIDDEN_DIMENSION: 10
    
    - EPOCHS: 20
   
    - LEARNING RATE: 0.001
    
    - LOSS FUNCTION: NLLLOSS
    
    - OPTIMIZER: ADAM